###1. Загрузка пакетов - библиотек

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU

###2. Загрузка данных

In [5]:
csv_url = "https://raw.githubusercontent.com/Daria-Chernykh/iad-lab1/refs/heads/main/V4.csv"
df = pd.read_csv(csv_url)


###3. Описательная статистика

In [6]:
df.head()

,Unnamed: 0,Customer Name,Customer e-mail,Country,Gender,Age,Annual Salary,Credit Card Debt,Net Worth,Car Purchase Amount
0,0,Martina Avila,cubilia.Curae.Phasellus@quisaccumsanconvallis.edu,USA,0.0,42,62812.09301,11609.380910,238961.2505,35321.45877
1,1,Harlan Barnes,eu.dolor@diam.co.uk,USA,0.0,41,66646.89292,9572.957136,530973.9078,45115.52566
2,2,Naomi Rodriquez,vulputate.mauris.sagittis@ametconsectetueradip...,USA,NaN,43,53798.55112,11160.355060,638467.1773,42925.70921
3,3,Jade Cunningham,malesuada@dignissim.com,USA,NaN,58,79370.03798,14426.164850,548599.0524,67422.36313
4,4,Cedric Leach,felis.ullamcorper.viverra@egetmollislectus.net,USA,1.0,57,59729.15130,5358.712177,560304.0671,55915.46248


Каждая строка представляет одного клиента, который потенциально заинтересован в покупке автомобиля.
Имеется 10 атрибутов:

1. Unnamed: 0 — порядковый номер записи;
2. Customer Name — имя клиента;
3. Customer e-mail — адрес электронной почты клиента;
4. Country — страна проживания клиента;
5. Gender — пол клиента (0 — мужчина, 1 — женщина, возможны пропуски);
6. Age — возраст клиента (в годах);
7. Annual Salary — годовой доход клиента;
8. Credit Card Debt — сумма задолженности по кредитной карте;
9. Net Worth — собственный капитал клиента (разница между активами и долгами);
10. Car Purchase Amount — целевой признак, отражающий ожидаемую сумму покупки автомобиля.

Таким образом, каждая строка описывает одного покупателя с финансовыми и демографическими характеристиками, на основании которых требуется построить регрессионную модель для прогноза значения Car Purchase Amount.

In [9]:
# Используем метод info( ) для получения краткого описания данных, в частности общего числа строк, а также типов и количества ненулевых значений каждого атрибута
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           500 non-null    int64  
 1   Customer Name        500 non-null    object 
 2   Customer e-mail      500 non-null    object 
 3   Country              500 non-null    object 
 4   Gender               244 non-null    float64
 5   Age                  500 non-null    int64  
 6   Annual Salary        500 non-null    float64
 7   Credit Card Debt     500 non-null    float64
 8   Net Worth            500 non-null    float64
 9   Car Purchase Amount  500 non-null    float64
dtypes: float64(5), int64(2), object(3)
memory usage: 39.2+ KB


В наборе данных 500 наблюдений и 10 признаков.  
Из них 7 числовых и 3 текстовых, включая категориальный признак Country.  

Все атрибуты набора данных, кроме Customer Name, Customer e-mail и Country, являются числовыми.
Эти три поля имеют тип object, поэтому могут содержать произвольные объекты Python, но поскольку данные были загружены из CSV-файла, ясно, что данные атрибуты являются текстовыми.

Атрибут Customer Name содержит имена клиентов,

Customer e-mail — адреса электронной почты,

а Country — страну проживания.

Значения в столбце Country повторяются, следовательно, этот признак является категориальным.
С помощью метода value_counts() можно определить, какие страны встречаются в наборе данных и сколько записей относится к каждой из них.

Остальные признаки — числовые:

* Gender — категориальный числовой признак (0 — мужчина, 1 — женщина), в нём имеются пропуски, которые следует обработать;
* Age — целочисленный признак, отражающий возраст клиента;
* Annual Salary — вещественный признак, годовой доход клиента;
* Credit Card Debt — вещественный признак, текущая задолженность по кредитной карте;
* Net Worth — вещественный признак, характеризующий собственный капитал (разница между активами и долгами);
* Car Purchase Amount — вещественный целевой признак, отражающий сумму покупки автомобиля, которую необходимо спрогнозировать.

Кроме перечисленных признаков, в наборе данных присутствует столбец Unnamed: 0, который представляет собой порядковый номер строки. Этот атрибут не несёт информативной нагрузки и будет удалён из дальнейшего анализа.

In [12]:
# Проверим значения признака "Country"
df["Country"].value_counts()

,count
Country,
USA,500


С помощью метода value_counts() видно, что все записи относятся к одной стране — USA.
Поскольку страна в данном наборе одна, признак Country не будет информативным и может быть исключён из дальнейшего анализа.
Таким образом, для построения модели можно оставить только числовые признаки и категориальный признак Gender (после его очистки и кодирования).

In [14]:
# Посмотрим сводку по числовым атрибутам
df.describe()

,Unnamed: 0,Gender,Age,Annual Salary,Credit Card Debt,Net Worth,Car Purchase Amount
count,500.000000,244.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,249.500000,0.500000,46.224000,62127.239608,9607.645049,431475.713625,44209.799218
std,144.481833,0.501028,7.990339,11703.378228,3489.187973,173536.756340,10773.178744
min,0.000000,0.000000,20.000000,20000.000000,100.000000,20000.000000,9000.000000
25%,124.750000,0.000000,41.000000,54391.977195,7397.515792,299824.195900,37629.896040
50%,249.500000,0.500000,46.000000,62915.497035,9655.035568,426750.120650,43997.783390
75%,374.250000,1.000000,52.000000,70117.862005,11798.867487,557324.478725,51254.709517
max,499.000000,1.000000,70.000000,100000.000000,20000.000000,1000000.000000,80000.000000


Метод describe() показал сводку по основным числовым признакам.
Количество непустых значений (count) для всех признаков, кроме Gender, составляет 500, что подтверждает отсутствие пропусков. Для Gender присутствуют пропущенные значения, так как count меньше 500.

Средние значения (mean) указывают, что средний возраст клиентов составляет примерно 45 лет,
средний годовой доход — около 65 000,
а средняя задолженность по кредитной карте — около 10–11 000.
Средний собственный капитал (Net Worth) находится в диапазоне 500–600 тыс.,
а средняя сумма покупки автомобиля (Car Purchase Amount) — около 45–50 тыс.

Показатель std (стандартное отклонение) показывает значительный разброс:
особенно велик разброс для признаков Net Worth и Car Purchase Amount, что говорит о сильной вариативности платежеспособности клиентов и стоимости приобретаемых автомобилей.

Значения 25%, 50% и 75% (процентили) показывают, что большая часть клиентов имеет доход между ~50 000 и ~80 000,
а собственный капитал большинства лежит между ~300 000 и ~700 000.
Таким образом, распределения признаков Annual Salary, Credit Card Debt и Net Worth сдвинуты вправо, что типично для финансовых данных.

Дополнительно стоит отметить, что минимальные и максимальные значения некоторых признаков являются округлёнными целыми числами,
что может указывать на предварительное усечение данных: значения ниже минимума и выше максимума могли быть приведены к граничным значениям.
Подобная особенность снижает вариативность данных на краях распределения и может незначительно повлиять на обучение модели, особенно при использовании методов, чувствительных к выбросам.